In [ ]:
""" 

This notebook pulls all POI's within a specified radius of each hotel using the Yelp API, calculates the mean nearby POI metrics for each hotel, 
then merges the results into a dataframe with the hotels dataframe

"""

In [86]:
# Imports

import pandas as pd
import requests
import os

In [ ]:
# Retreiving API key from environment variable
key = os.environ.get('api_key_places')

In [88]:
# For Sam's API key, comment out

#from dotenv import load_dotenv
#load_dotenv()
#key = os.getenv('YELP_API_KEY3')

In [89]:
# Importing Yelp hotels dataframe

df_hotels = pd.read_csv('yelp_hotels.csv')
df_hotels.head()

,hotel_id,hotel_name,hotel_review_count,hotel_categories,hotel_rating,hotel_price,hotel_distance,hotel_latitude,hotel_longitude,hotel_address1,hotel_city,hotel_zip_code,hotel_country,hotel_state,hotel_display_address
0,yeesT9W64tpVq5Gcae18Sg,Delta Hotels by Marriott Toronto,161,hotels,4.0,2,3623.820220,43.643069,-79.383865,75 Lower Simcoe Street,Toronto,M5J 3A6,CA,ON,"['75 Lower Simcoe Street', 'Toronto, ON M5J 3A..."
1,kKnGY0Xj7dG4ubjrbygynw,Residence Inn by Marriott Toronto Downtown,95,hotels,4.0,2,3320.732218,43.644790,-79.390530,255 Wellington Street W,Toronto,M5V 3P9,CA,ON,"['255 Wellington Street W', 'Toronto, ON M5V 3..."
2,XHrPQKUTJXJGPH9etKUYUw,Courtyard by Marriott Toronto Downtown,152,hotels,3.0,2,1718.071430,43.662410,-79.383149,475 Yonge Street,Toronto,M4Y 1X7,CA,ON,"['475 Yonge Street', 'Toronto, ON M4Y 1X7', 'C..."
3,zYRBRfYuq_6x-wNKa8NqrA,Le Germain Hotel,55,hotels,4.5,3,3202.097393,43.645751,-79.391025,30 Mercer Street,Toronto,M5V 1H3,CA,ON,"['30 Mercer Street', 'Toronto, ON M5V 1H3', 'C..."
4,PCkJAv7WQJ08C1RA6ilqWA,1 Hotel Toronto,5,hotels,5.0,2,3517.449544,43.642826,-79.401439,550 Wellington Street W,Toronto,M5V 2V4,CA,ON,"['550 Wellington Street W', 'Toronto, ON M5V 2..."


In [91]:
# Definging the function to find all POI's of a specific category within a set distance from any hotels, using the Yelp API again

def find_POI_yelp(hotel_ids,latitudes,longitudes,radius,limit,categories):
    
    #fields = 'fsq_id,name,rating,popularity,price'
    list_POI = []
    
    # Find all POI's within 1000m of each bike station.  This will return one column with station_id and one column containing lists of POI's.
    for hotel_id,lat,long in zip(hotel_ids,latitudes,longitudes):
        url = 'https://api.yelp.com/v3/businesses/search'
        params = {
            'limit': limit,
            'latitude': lat,
            'longitude': long,
            'term': categories,
            'categories': categories,
            'radius': radius
        }

        headers = {"accept": "application/json","Authorization": 'bearer ' + key}
        response = requests.get(url, headers=headers,params=params).json()['businesses']
        list_POI.append([hotel_id,response])
    
    # convert to dataframe
    df = pd.DataFrame(list_POI,columns=['hotel_id','businesses'])

    # create 1 row for each match of bike station to POI.  There are duplicates of both bike stations and POI's, but each matchup is unique.
    df = df.explode('businesses').reset_index(drop=True) 
    df = pd.concat([df,pd.json_normalize(df['businesses'])],axis=1).drop('businesses',axis=1)
    
    return df
    #return requests.get(url, headers=headers,params=params).json()#['businesses']


In [92]:
# Running the function to find all POI's within a certain radius of every hotel

n = 200 # number of hotels to use
r = 150 # radius in metres
limit = 40 # max number of POI's to return for each hotel
categories = 'restaurants' # POI description

df = df_hotels.head(n)

df_POI = find_POI_yelp(df['hotel_id'],df['hotel_latitude'],df['hotel_longitude'],r,limit,categories)

In [94]:
# Check df_POI shape

df_POI.shape

(2736, 25)

In [95]:
# View first 5 rows of df_POI

df_POI.head()

,hotel_id,id,alias,name,image_url,is_closed,url,review_count,categories,rating,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
0,yeesT9W64tpVq5Gcae18Sg,9eu2lQMROTKdrPyGkwZLYw,soco-kitchen-bar-toronto,SOCO Kitchen + Bar,https://s3-media3.fl.yelpcdn.com/bphoto/85--in...,False,https://www.yelp.com/biz/soco-kitchen-bar-toro...,236.0,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",3.5,...,43.642653,-79.383847,75 Lower Simcoe Street,,,Toronto,M5J 3A6,CA,ON,"[75 Lower Simcoe Street, Toronto, ON M5J 3A6, ..."
1,yeesT9W64tpVq5Gcae18Sg,x0MaTw6He1w_1nddeSJU2Q,the-roof-at-soco-toronto,The Roof at SOCO,https://s3-media2.fl.yelpcdn.com/bphoto/gF-WNQ...,False,https://www.yelp.com/biz/the-roof-at-soco-toro...,17.0,"[{'alias': 'newcanadian', 'title': 'Canadian (...",4.0,...,43.643030,-79.383790,75 Lower Simcoe Street,Fourth Floor,,Toronto,M5J 3A6,CA,ON,"[75 Lower Simcoe Street, Fourth Floor, Toronto..."
2,yeesT9W64tpVq5Gcae18Sg,D1lAVtlav4atQTJnIvtcpw,taverna-mercatto-toronto,Taverna Mercatto,https://s3-media2.fl.yelpcdn.com/bphoto/I8Guuj...,False,https://www.yelp.com/biz/taverna-mercatto-toro...,229.0,"[{'alias': 'italian', 'title': 'Italian'}]",3.5,...,43.642672,-79.382760,120 Bremner Boulevard,,,Toronto,M5J 3A6,CA,ON,"[120 Bremner Boulevard, Toronto, ON M5J 3A6, C..."
3,yeesT9W64tpVq5Gcae18Sg,9-Lipa9HiHL_EAfeDfPq1g,busters-sea-cove-toronto-3,Buster's Sea Cove,https://s3-media3.fl.yelpcdn.com/bphoto/4jrvkf...,False,https://www.yelp.com/biz/busters-sea-cove-toro...,33.0,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",3.5,...,43.642566,-79.383051,120 Bremner Boulevard,,,Toronto,M5J 0A1,CA,ON,"[120 Bremner Boulevard, Toronto, ON M5J 0A1, C..."
4,yeesT9W64tpVq5Gcae18Sg,JK_EaeYRYp3oCsc3gewuRQ,thai-express-toronto-18,Thai Express,https://s3-media4.fl.yelpcdn.com/bphoto/PMRtxd...,False,https://www.yelp.com/biz/thai-express-toronto-...,5.0,"[{'alias': 'thai', 'title': 'Thai'}]",2.5,...,43.643001,-79.383354,120 Bremner Boulevard,,,Toronto,M5J 0B2,CA,ON,"[120 Bremner Boulevard, Toronto, ON M5J 0B2, C..."


In [96]:
# Viewing columns in df_POI

df_POI.columns

Index(['hotel_id', 'id', 'alias', 'name', 'image_url', 'is_closed', 'url',
       'review_count', 'categories', 'rating', 'transactions', 'price',
       'phone', 'display_phone', 'distance', 'coordinates.latitude',
       'coordinates.longitude', 'location.address1', 'location.address2',
       'location.address3', 'location.city', 'location.zip_code',
       'location.country', 'location.state', 'location.display_address'],
      dtype='object')

In [97]:
# Deciding which columns to keep in df_POI

df_POI = df_POI[['hotel_id', 'id', 'name',
       'review_count', 'categories', 'rating', 'transactions', 'price',
       'distance', 'coordinates.latitude',
       'coordinates.longitude', 'location.address1', 
       'location.city', 'location.zip_code',
       'location.country', 'location.state', 'location.display_address']]

In [98]:
# Renaming columns in df_POI

df_POI.columns = ['hotel_id', 'poi_id', 'poi_name',
       'poi_review_count', 'poi_categories', 'poi_rating', 'poi_transactions', 'poi_price',
       'poi_distance', 'poi_latitude',
       'poi_longitude', 'poi_address1', 
       'poi_city', 'poi_zip_code',
       'poi_country', 'poi_state', 'poi_display_address']

In [99]:
# Creating new poi_price column that turns the number of dollar signs into integers

df_POI['poi_price_int'] = df_POI['poi_price'].str.len()

In [101]:
# Saving df_POI to .csv file

df_POI.to_csv('df_POI.csv')

In [102]:
# Creating a pivot table that groups df_POI by hotel_id and aggregates the means for price, rating, and review counts for the POI's nearby each hotel.  It also aggregates the count of POI's near each hotel.

pivot_poi = df_POI.pivot_table(index='hotel_id',values=['poi_review_count','poi_rating','poi_price_int'],
                               aggfunc=['mean','count']).reset_index()

pivot_poi.columns = ['hotel_id','mean_poi_price','mean_poi_rating','mean_poi_review_count','count_poi_price','count_poi','count_poi_review_count']
pivot_poi = pivot_poi[['hotel_id','mean_poi_price','mean_poi_rating','mean_poi_review_count','count_poi']]


pivot_poi.head()		

,hotel_id,mean_poi_price,mean_poi_rating,mean_poi_review_count,count_poi
0,-8oIw0-YzvYNRydiOPyasw,NaN,3.000000,5.000000,1
1,-zge19VgbAf9FZzVu21vkg,2.666667,3.333333,294.333333,3
2,0UEQvzlePYUnpe_a40-RtQ,1.000000,4.500000,19.000000,1
3,1VMccrsa7R7nzDR37HksKA,2.000000,3.000000,4.000000,1
4,1YyhRc6HC101_ZrA3h7pEg,NaN,2.000000,1.000000,1


In [103]:
# Merging df_hotles with pivot_poi

df_merge = df_hotels.merge(pivot_poi,on='hotel_id', how='left')
df_merge.head()

,hotel_id,hotel_name,hotel_review_count,hotel_categories,hotel_rating,hotel_price,hotel_distance,hotel_latitude,hotel_longitude,hotel_address1,hotel_city,hotel_zip_code,hotel_country,hotel_state,hotel_display_address,mean_poi_price,mean_poi_rating,mean_poi_review_count,count_poi
0,yeesT9W64tpVq5Gcae18Sg,Delta Hotels by Marriott Toronto,161,hotels,4.0,2,3623.820220,43.643069,-79.383865,75 Lower Simcoe Street,Toronto,M5J 3A6,CA,ON,"['75 Lower Simcoe Street', 'Toronto, ON M5J 3A...",1.800000,2.766667,58.133333,15
1,kKnGY0Xj7dG4ubjrbygynw,Residence Inn by Marriott Toronto Downtown,95,hotels,4.0,2,3320.732218,43.644790,-79.390530,255 Wellington Street W,Toronto,M5V 3P9,CA,ON,"['255 Wellington Street W', 'Toronto, ON M5V 3...",1.750000,2.925926,53.111111,27
2,XHrPQKUTJXJGPH9etKUYUw,Courtyard by Marriott Toronto Downtown,152,hotels,3.0,2,1718.071430,43.662410,-79.383149,475 Yonge Street,Toronto,M4Y 1X7,CA,ON,"['475 Yonge Street', 'Toronto, ON M4Y 1X7', 'C...",1.666667,3.060000,71.360000,25
3,zYRBRfYuq_6x-wNKa8NqrA,Le Germain Hotel,55,hotels,4.5,3,3202.097393,43.645751,-79.391025,30 Mercer Street,Toronto,M5V 1H3,CA,ON,"['30 Mercer Street', 'Toronto, ON M5V 1H3', 'C...",2.238095,2.955882,96.558824,34
4,PCkJAv7WQJ08C1RA6ilqWA,1 Hotel Toronto,5,hotels,5.0,2,3517.449544,43.642826,-79.401439,550 Wellington Street W,Toronto,M5V 2V4,CA,ON,"['550 Wellington Street W', 'Toronto, ON M5V 2...",2.125000,2.772727,41.545455,22


In [107]:
# Percentage of Yelp columns that are not null

print(df_merge.count()/df_merge.shape[0])
print()
print(str(df_merge.shape[0]) + ' rows')
print('Mean not null percentage: '+ str((df_merge.count()/df_merge.shape[0]).mean()))

hotel_id                 1.0
hotel_name               1.0
hotel_review_count       1.0
hotel_categories         1.0
hotel_rating             1.0
hotel_price              1.0
hotel_distance           1.0
hotel_latitude           1.0
hotel_longitude          1.0
hotel_address1           1.0
hotel_city               1.0
hotel_zip_code           1.0
hotel_country            1.0
hotel_state              1.0
hotel_display_address    1.0
mean_poi_price           1.0
mean_poi_rating          1.0
mean_poi_review_count    1.0
count_poi                1.0
dtype: float64

200 rows
Mean not null percentage: 1.0


In [106]:
# Filling null values in numeric columns with the mean from the respective column

df_merge['mean_poi_price'] = df_merge['mean_poi_price'].fillna(df_merge['mean_poi_price'].mean())
df_merge['mean_poi_rating'] = df_merge['mean_poi_rating'].fillna(df_merge['mean_poi_rating'].mean())
df_merge['mean_poi_review_count'] = df_merge['mean_poi_review_count'].fillna(df_merge['mean_poi_review_count'].mean())

# Replacing nulls with blanks in categorical data
df_merge['hotel_address1'] = df_merge['hotel_address1'].fillna('')
df_merge['hotel_zip_code'] = df_merge['hotel_zip_code'].fillna('')

In [109]:
# Writing df_merge to a .csv file
df_merge.to_csv('hotels_and_POIs.csv', index=False)